In [22]:
import sys
import soundcloud
import networkx as nx

In [23]:
from functools import partial
from utils import get_results, handle_http_errors

In [24]:
def getAttr(resource, attr):
    if hasattr(resource, attr): return getattr(resource, attr)
    return None

getUsername = partial(getAttr, attr='username')
getid = partial(getAttr, attr='id')
getUserid = partial(getAttr, attr='user_id')

In [25]:
def profile2id(profile):
    print("get user id result is {}".format(getUserid(profile)))
    print("get id result is {}".format(getid(profile)))
    return getUserid(profile) if isinstance(getUserid(profile), int) else getid(profile)

In [26]:
def profile2username(profile):
    return getUsername(profile) if isinstance(getUsername(profile), str) else str(getid(profile))

In [27]:
# A global profile graph used to iterate through the various algorithms.
# Each node is profile id, with edges weighted by activity between then.
profile_graph = nx.MultiDiGraph()


In [28]:

client = soundcloud.Client(client_id='454aeaee30d3533d6d8f448556b50f23')
name = input("Enter a soundcloud artist to analyze: ")

Enter a soundcloud artist to analyze: Sybyr


In [29]:
# Artist of interest

search = client.get('/users/', q = name)[0]

In [30]:
search.__dict__

{'obj': {'avatar_url': 'https://i1.sndcdn.com/avatars-cIS3ELGYY7j2e0q9-OEySHA-large.jpg',
  'id': 13377746,
  'kind': 'user',
  'permalink_url': 'https://soundcloud.com/sybyr',
  'uri': 'https://api.soundcloud.com/users/13377746',
  'username': 'Sybyr',
  'permalink': 'sybyr',
  'last_modified': '2020/08/16 23:02:35 +0000',
  'first_name': 'Sybyrmusic.com',
  'last_name': '',
  'full_name': 'Sybyrmusic.com',
  'city': '',
  'description': 'bookings/email: sybyrcentral@gmail.com\nAnti-World: http://antiworld.co (@antiworldglobe) on all networks\n(Prod. by Landfill) at http://prodbylandfill.com',
  'country': 'United States',
  'track_count': 437,
  'public_favorites_count': 0,
  'followers_count': 52157,
  'followings_count': 23,
  'plan': 'Pro Unlimited',
  'myspace_name': None,
  'discogs_name': None,
  'website_title': 'click Here',
  'website': 'http://antiworld.lnk.to/SYBYR',
  'reposts_count': 104,
  'comments_count': 110,
  'online': False,
  'likes_count': 0,
  'playlist_count':

In [31]:
print("Artist interpreted as: %s" % getUsername(search))

Artist interpreted as: Sybyr


In [32]:
uid2username_cache = {}
tid2username_cache = {}
cid2username_cache = {}
tid2userid_cache = {}
#commentid2userid_cache = {}

# Creating Decorator: handle_http_errors

In [33]:
@handle_http_errors
def userid2username(uid):
    global uid2username_dict
    username = uid2username_cache.get(uid, None)
    if username is not None: 
        return username
    # username is none, we don't have it in cache
    
    uid_str = str(uid)
    get = '/users/{}'.format(uid_str)
    result = client.get(get)
    username = result.username
    uid2username_cache[uid] = username
    return username

def trackid2username(tid):
    global tid2username_dict
    username = tid2username_cache.get(uid, None)
    if username is not None: 
        return username
    # username is none, we don't have it in cache
    
    tid_str = str(tid)
    get = '/tracks/{}'.format(tid_str)
    result = client.get(get)
    username = result.user['username']
    tid2username_cache[tid] = username
    return username

def commentid2username(cid):
    global cid2username_dict
    username = cid2username_cache.get(uid, None)
    if username is not None: 
        return username
    # username is none, we don't have it in cache
    
    cid_str = str(cid)
    get = '/comments/{}'.format(cid_str)
    result = client.get(get)
    username = result.user['username']
    cid2username_cache[cid] = username
    return username
    
def trackid2userid(tid):
    global tid2userid_dict
    userid = tid2userid_cache.get(tid, None)
    if userid is not None: 
        return userid
    # username is none, we don't have it in cache
    
    tid_str = str(tid)
    get = '/tracks/{}'.format(tid_str)
    result = client.get(get)
    userid = result.user['id']
    tid2userid_cache[tid] = userid
    return userid
    

# get social data functions

In [34]:
def getRelationships(profile, client, url): 
    return get_results(client, url)

@handle_http_errors
def getFollows(profile):
    # get all profiles a user follows
    follows = get_results(client, '/users/{0:s}/followings/'.format(str(profile)))
    return follows

@handle_http_errors
def getFollowers(profile):
    # get all profiles that follow a user
    followers = get_results(client, '/users/{0:s}/followers/'.format(str(profile)))
    return followers

@handle_http_errors
def getUserFavorites(profile):
    user_favorites = get_results(client, '/users/{0:s}/favorites/'.format(str(profile)))
    return user_favorites

@handle_http_errors
def getUserComments(profile):
    comments = get_results(client, '/users/{0:s}/comments/'.format(str(profile)))
    return comments

@handle_http_errors
def getTracks(profile):
    tracks = get_results(client, '/users/{0:s}/tracks/'.format(str(profile)))
    return tracks

@handle_http_errors
def getTrackComments(track):
    track_comments = get_results(client, '/tracks/{0:s}/comments/'.format(str(track)))
    return track_comments

@handle_http_errors
def getTrackFavoriters(track):
    track_favoriters = get_results(client, '/tracks/{0:s}/favoriters/'.format(str(track)))
    return track_favoriters

# build graph functions

In [35]:
def addProfileNx(profile, profileGraph):
    profileGraph.add_node(profile)
    #return profileGraph

def addPairNx(profile, neighbor, profileGraph):
    addProfileNx(profile)
    addProfileNx(neighbor)

def addFollowsNx(profile, follows, profileGraph):
    for follow in follows:
        # enforce that already existing follows don't get added again
        if not profileGraph.has_edge(profile, follow, 'soundcloud_follow'):
            profileGraph.add_edge(profile, follow, 'soundcloud_follow',weight=1)

# TODO: enforce that the same favorite isn't counted twice
# easy solve: keep a list of tracks as an edge attribute
def addUserFavoritesNx(profile, user_favorites, profileGraph):
    for user_favorite in user_favorites:
        if profileGraph.has_edge(profile, user_favorite, 'soundcloud_favorite'):
            profileGraph.edges[profile, user_favorite,'soundcloud_favorite']['weight']+=1
        else:
            profileGraph.add_edge(profile, user_favorite, 'soundcloud_favorite',weight=1)

# TODO: enforce that the same comment isn't counted twice
# easy solve: keep a dictionary of track-comments pairs as an edge attribute
def addUserCommentsNx(profile, user_comments, profileGraph):
    for user_comment in user_comments:
        if profileGraph.has_edge(profile, user_comment, 'soundcloud_comment'):
            profileGraph.edges[profile, user_comment,'soundcloud_comment']['weight']+=1
        else:
            profileGraph.add_edge(profile, user_comment, 'soundcloud_comment',weight=1)

def addFollowersNx(profile, followers, profileGraph):
    for follower in followers:
        if not profileGraph.has_edge(follower, profile, 'soundcloud_follow'):
            profileGraph.add_edge(follow, profile, 'soundcloud_follow',weight=1)

# print graph functions

In [41]:
def printGraph(G):
    for profile, neighbor, key, weight in G.edges(data=True,keys=True):
        print("Profile: {}".format(userid2username(profile)))
        print("Neighbor: {}".format(userid2username(neighbor)))
        print(key)
        print(weight)
#     for profile in G.nodes():
#         if profile:
#             try:
#                 profile_name = id2username(profile)
#                 print("\t", "Printing: %s (%s)" % (profile_name, profile))
#                 # why does successors return an iterator instead of a list? getting fancy for no reason
#                 follows = [follow for follow in G.successors(profile)]
#                 followers = [follower for follower in G.predecessors(profile)]
#                 try:    
#                     print("\t", profile_name + " has " + str(len(follows)) + " followings")
#                     print("\t", profile_name + " follows " + ", ".join(map(lambda x: id2username(x), follows)))
#                 except TypeError: 
#                     print("No follows here!")
#                 try:    
#                     print("\t", profile_name + " has " + str(len(followers)) + " followers")
#                     print("\t", profile_name + " is followed by " + ", ".join(map(lambda x: id2username(x), followers)))
#                 except TypeError:
#                     print("No followers here!")
#                     print("-"*40)
#             except UnicodeError:
#                 print("Artist's username not found" )   

# 1b. Testing Functions

In [37]:
# list of profiles to query
profiles_to_query = [search.id]
profiles_to_query

[13377746]

In [38]:
depth = 1
i = 0

# list of profiles we could not query
unavailable_profiles = []

Only uncomment the code in the loop if you are willing to wait a while to run the cell below:

In [39]:
for t in range(depth):
    print("Iteration " + str(t))
    profiles_to_query = list(set(profiles_to_query))
    
    for profile in profiles_to_query:
        username = userid2username(profile)
        if username:
            print("\t", "Enqueueing: %s (%s)" % (username, profile))
            addProfileNx(profile,profile_graph)

#             encountered_follows = getFollows(profile)
#             encountered_follow_profiles = [follow.id for follow in encountered_follows]
#             print("encountered follows: " + ", ".join([id2username(profile) for profile in encountered_follow_profiles]))
#             print('\n')
#             addFollowsNx(profile, encountered_follow_profiles, profile_graph)

#             encountered_user_favorites = getUserFavorites(profile)
#             encountered_user_favorite_profiles = [getUserid(track) if isinstance(getUserid(track), int) else track.user_id for track in encountered_user_favorites]
#             print("encountered user favorite profiles: " + ", ".join([id2username(profile) for profile in encountered_user_favorite_profiles]))
#             print('\n')
#             addUserFavoritesNx(profile, encountered_user_favorite_profiles, profile_graph)

            encountered_user_comments = getUserComments(profile)
            encountered_user_comment_profiles = [trackid2userid(comment.track_id) for comment in encountered_user_comments]
            print("encountered user comment profiles: " + ", ".join([userid2username(profile) for profile in encountered_user_comment_profiles]))
            addUserCommentsNx(profile, encountered_user_comment_profiles, profile_graph)
            print('\n')

#             encountered_followers = getFollowers(profile)
#             encountered_follower_profiles = [follower.id for follower in encountered_followers]
#             print("encountered followers: " + ", ".join([id2username(profile) for profile in encountered_follower_profiles]))
#             print('\n')
#             addFollowersNx(profile, encountered_follower_profiles, profile_graph)

#             encountered_tracks = getTracks(profile)

#             encountered_track_comments = []
#             encountered_track_favoriters = []

#             for track in encountered_tracks:
#                 print("getting comments / favoriters from user track, {}".format(track.title))
#                 encountered_track_comments += getTrackComments(track.id)
#                 encountered_track_favoriters += getTrackFavoriters(track.id)

#             print(encountered_track_comments[0].__dict__)
#             print(encountered_track_favoriters[0].__dict__)

#             print("encountered track comments: " + ", ".join([getUsername(comment) if isinstance(getUsername(comment), str) else comment.user['username'] for comment in encountered_track_comments]))#([str(comment.__dict__) for comment in encountered_track_comments]))#
#             print("encountered track favoriters: " + ", ".join([getUsername(favoriter) if isinstance(getUsername(favoriter), str) else favoriter.user['username'] for favoriter in encountered_track_favoriters]))
        else:
            print("\t", "Artist ID %s is not query-able" % profile)
            unavailable_profiles.append(profile)            

Iteration 0
	 Enqueueing: Sybyr (13377746)
encountered user comment profiles: Sybyr, ERIC NORTH (@RI0T_ANGEL), Anti-World Fan Radio / A.k.a YellowVision, Sybyr, MITCHELL BAY, Original God, SPACE MAGIC スペース マジック




In [42]:
printGraph(profile_graph)

Profile: Sybyr
Neighbor: Sybyr
soundcloud_comment
{'weight': 2}
Profile: Sybyr
Neighbor: ERIC NORTH (@RI0T_ANGEL)
soundcloud_comment
{'weight': 1}
Profile: Sybyr
Neighbor: Anti-World Fan Radio / A.k.a YellowVision
soundcloud_comment
{'weight': 1}
Profile: Sybyr
Neighbor: MITCHELL BAY
soundcloud_comment
{'weight': 1}
Profile: Sybyr
Neighbor: Original God
soundcloud_comment
{'weight': 1}
Profile: Sybyr
Neighbor: SPACE MAGIC スペース マジック
soundcloud_comment
{'weight': 1}


# 2. metadata functions

In [14]:
def getUserInfo(profile):
    info = ['id','permalink', 'username', 'avatar_url', 'country', 'city', 
            'website', 'track_count', 'followers_count', 'followings_count']
    return {i: getAttr(profile, i) for i in info}

def getTrackInfo(track):
    info = ['id', 'user_id', 'created_at', 'streamabale', 'downloadable',
            'playback_count', 'download_count', 'favoritings_count','comment_count']
    return {i: getAttr(track, i) for i in info}

def getUserFavInfo(fav):
    info = ['id', 'user_id','last_modified']
    return {i: getAttr(fav, i) for i in info}

def getTrackFavInfo(fav):
    info = ['id', 'username', 'last_modified']
    return {i: getAttr(fav, i) for i in info}

def getUserCommInfo(comm):
    info = ['id', 'user_id', 'track_id', 'timestamp']
    return {i: getAttr(comm, i) for i in info}

def getTrackCommInfo(comm):
    info = ['id', 'user_id', 'track_id','timestamp']
    return {i: getAttr(comm, i) for i in info}